In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import numpy as np
import pandas as pd

def evaluate(X, y):
    pd.concat([X, y], axis = 1)
    train, valid_plus_test = train_test_split(X, random_state=2021, test_size=0.3)
    val, test = train_test_split(valid_plus_test, random_state=2021, test_size=0.67)
    lgb_train = lgb.Dataset(train.iloc[:, :-1].values, 
                        train.iloc[:, -1].values, free_raw_data=False)
    lgb_val = lgb.Dataset(val.iloc[:, :-1].values, 
                        val.iloc[:, -1].values, free_raw_data=False)
    lgb_test = lgb.Dataset(test.iloc[:, :-1].values, 
                        test.iloc[:, -1].values, free_raw_data=False)
    params = {
        "objective" : "mse",
        "metric" : "mae",
        "max_depth" : 7,
        "num_leaves" : 100,
        "learning_rate" : 0.05,
        "bagging_fraction" : 0.3,
        "feature_fraction" : 0.15,
        "lambda_l1" : 5,
        "lambda_l2" : 5,
        "bagging_seed" : 42,
        "verbosity" : 0,
        "seed": 42
    }
    
    clf = lgb.train(
        params,
        lgb_train,
        valid_sets=[lgb_val],
        num_boost_round=500,
        early_stopping_rounds=10,
        verbose_eval=10
    )
    
    pred1 = clf.predict(train.iloc[:, :-1].values)
    sc1 = mean_absolute_error(train.iloc[:, -1].values, pred1)
    pred2 = clf.predict(val.iloc[:, :-1].values)
    sc2 = mean_absolute_error(val.iloc[:, -1].values, pred2)
    pred3 = clf.predict(test.iloc[:, :-1].values)
    sc3 = mean_absolute_error(test.iloc[:, -1].values, pred3)
    return (sc1, sc2, sc3)